In [ ]:
import torch

class Rnn_layer:
    def __init__(self,vocab_size,hidden_size):
        self.wh=torch.randn(hidden_size,hidden_size,requires_grad=True)
        self.wx=torch.randn(vocab_size,hidden_size,requires_grad=True)
        self.wy=torch.rand(hidden_size,vocab_size,requires_grad=True)
        self.by=torch.randn(1,vocab_size,requires_grad=True)
        self.bh=torch.randn(1,hidden_size,requires_grad=True)
        self.ht_1=torch.zeros(1,hidden_size)
    def hiden_satate(self,xt):
        hiden_state= self.wh*self.ht_1+xt*self.wx+self.bh
        self.ht_1=hiden_state
        return hiden_state
    def output(self,hiden_state):
        return hiden_state*self.wy+self.by

In [ ]:
from  Rnn_operations import *
data='messi'
vocabulaire=tri_gram_vocab()
tokenized_data=tokenize(vocabulaire,data)
embdings=get_embedings(tokenized_data,vocabulaire,5)


In [ ]:
class Encoder:
   def __init__(self):
      self.layers=[]
      self.parameters=[]
      self.states=[]

   def add_layer(self,vocab_size,hidden_size):
        layer=Rnn_layer(vocab_size,hidden_size)
        self.layers.append(
            layer
        )
        self.parameters.append(layer.parameters)
   
   def train(self,len_seq,embedings,):
         for t in len(len_seq):
            e=embedings[t]
            for layer in self.layers:
               ht=layer.hiden_state(e)
               self.states.append(ht)
               
class Decoder:
    def __init__(self,attention_size,hidden_size,vocab_size):
        self.loss=torch.nn.CrossEntropyLoss()
        self.aprameters=[]
        self.optimize=torch.optim.Adam()
        self.w_s=torch.randn(attention_size,hidden_size,requires_grad=True)
        self.w_h=torch.randn(attention_size,hidden_size,requires_grad=True)
        self.v_a=torch.randn(attention_size,biais=False,requires_grad=True)
        self.encoder_states=[]
        self.st=None
        self.wy=torch.rand(hidden_size,vocab_size,requires_grad=True)
        self.by=torch.randn(1,vocab_size,requires_grad=True)

    def linear(self,hiden_state):
        return hiden_state*self.wy+self.by

    def get_encoder_sates(self,states):
        self.encoder_states=[states]
        st=self.encoder_states[-1]
        return st,self.encoder_states
    def socre(self):
        e= self.v*torch.tanh(self.w_s@self.encoder_states+self.w_h@self.st)
        a=torch.softmax(e)
        return torch.sum(a@self.encoder_states)
    def add_layer(self,vocab_size,hidden_size):
        layer=Rnn_layer(vocab_size,hidden_size)
        self.layers.append(
            layer
        )
        self.parameters.append(layer.parameters)
   
    def train(self,len_seq,ct,seq,ytrain):
        for epoch in range(100):
         for t in len(len_seq):
            xt=seq[t]
            st_1=self.st
            for layer in self.layers:
               ht=layer.hiden_state(st_1,torch.concat(xt,ct))
               xt=ht
            self.st=xt
            input=torch.concat(ct,self.st)
            ypred=self. linear(input)  
            ypred=layer.output(xt)
            l=self.critrion(ypred,ytrain[t])
            self.optimizer.zero_grad()
            with torch.no_grad():
               l.backward()
               self.optimize.step()
            loss+=l
         print(f"{epoch} loss :{loss/len(len_seq)}")
                   